In [1]:
from google.cloud import storage
import pyarrow.parquet as pq
import pandas as pd
import json

In [2]:
storage_client = storage.Client()

In [3]:
def read_parquet_from_gcs(bucket_name, file_path):

    # Especifica la ruta completa del archivo en GCS
    gcs_path = f'gs://{bucket_name}/{file_path}'

    # Lee directamente desde GCS sin descargar localmente
    table = pq.read_table(gcs_path)

    # (Opcional) Convierte los datos a DataFrame de Pandas
    df = table.to_pandas()

    return df

In [7]:
bucket_name = 'project_yelp_parquet'
file_path_user = 'metadata-sitios.parquet'
df_metadata_sitios = read_parquet_from_gcs(bucket_name, file_path_user)

In [8]:
duplicates_review = df_metadata_sitios.duplicated(['name', 'address', 'latitude', 'longitude']).sum()
print(duplicates_review)

27072


In [9]:
df_metadata_sitios.sample(10)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
1931289,Granitex Construction Co,"Granitex Construction Co, 185 Paularino Ave # ...",0x80dcdee226be6ea9:0xfbab959e2e8b5766,None,33.680491,-117.876128,[Construction company],5.0,2,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x80dcdef643a55c5f:0x613830d87b8ea874, 0x80dc...",https://www.google.com/maps/place//data=!4m2!3...
297575,Dorsett Chiropractic,"Dorsett Chiropractic, 11705 Dorsett Rd, Maryla...",0x87df320d2dd998df:0xa57684ed7c980d37,None,38.715211,-90.428244,"[Chiropractor, Department of Transportation, N...",5.0,7,None,"[[Friday, 9AM–5PM], [Saturday, Closed], [Sunda...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 9AM Mon,"[0x87df31940629ca55:0xfe9252c6030ee278, 0x87df...",https://www.google.com/maps/place//data=!4m2!3...
1216006,SERVER REPAIR (Technical and Onsite Support),None,0x876c7df4c54e8e23:0x9795ab07886f094,None,39.767937,-104.895725,"[Computer support and services, Service establ...",5.0,6,None,"[[Thursday, 10AM–10PM], [Friday, 10AM–10PM], [...",None,Closed ⋅ Opens 10AM,"[0x876c7d7f38eda0f3:0x8f37484b83105777, 0x876c...",https://www.google.com/maps/place//data=!4m2!3...
3010882,Solar Nails II,"Solar Nails II, 1115 N Main St, Cleburne, TX 7...",0x864e4654f77eec71:0xae440d2bcf393eb6,None,32.362338,-97.392426,"[Beauty salon, Nail salon]",4.4,93,None,"[[Monday, 10AM–6PM], [Tuesday, 10AM–6PM], [Wed...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x864e47c1af3227f3:0xef82e2c1141d3d80, 0x864e...",https://www.google.com/maps/place//data=!4m2!3...
658222,Magellan Marine International LLC,"Magellan Marine International LLC, 2816 Athani...",0x8620b0236ef5c97f:0x8715e1b7b97299de,None,30.001859,-90.157475,[Consultant],5.0,1,None,None,None,None,"[0x8620a61e9738d00f:0xbcc3782c4f8c9920, 0x8620...",https://www.google.com/maps/place//data=!4m2!3...
2959588,Fairfax Presbyterian Church,"Fairfax Presbyterian Church, 10723 Main St, Fa...",0x89b64eee42ca1423:0x51f1b5eb13dbda88,None,38.846877,-77.317572,"[Presbyterian church, Church]",4.8,18,None,"[[Tuesday, Closed], [Wednesday, 9AM–2PM], [Thu...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 9AM Wed,"[0x89b64c48aa243cb1:0xda7b75da2f65adb9, 0x89b6...",https://www.google.com/maps/place//data=!4m2!3...
1002139,Pray4MeAuto2,"Pray4MeAuto2, 1825 S Cumberland St, Morristown...",0x885b972cfee4ec17:0x5c0e06488c518a3b,None,36.196626,-83.279010,"[Used tire shop, Mechanic, Oil change service]",4.1,18,None,"[[Monday, 11AM–7PM], [Tuesday, 11AM–7PM], [Wed...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 11AM Tue,None,https://www.google.com/maps/place//data=!4m2!3...
1914138,Salon G'onna,"Salon G'onna, 100 Swiss Way, Elizabeth, PA 15037",0x8834e41b7bf9bb89:0xac67a31436d2aad5,None,40.281020,-79.835404,"[Hair salon, Beauty salon]",4.5,18,None,"[[Saturday, 9AM–3PM], [Sunday, Closed], [Monda...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 9AM,"[0x8834e49d4ddb515b:0xe898a8f7bfb04a11, 0x8834...",https://www.google.com/maps/place//data=!4m2!3...
3024423,Texas Health and Human Services Commission,"Texas Health and Human Services Commission, 10...",0x86f07c1ecc1c0cc1:0xb6742a2e32bbc663,None,30.884759,-102.878388,[Social services organization],4.5,8,None,"[[Monday, 8AM–5PM], [Tuesday, 8AM–5PM], [Wedne...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 5PM,"[0x86f07c1ec52bc365:0x3b493e7fe77bd6f1, 0x86f0...",https://www.google.com/maps/place//data=!4m2!3...
1801241,Santa Fe Artists Market,"Santa Fe Artists Market, 3201-C Zafarano Drive...",0x1526b74ed894a42f:0x4919aad3a61cf9dd,None,35.683345,-105.948748,[Art gallery],4.6,8,None,"[[Monday, Closed], [Tuesday, Closed], [Wednesd...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 8AM Sat,"[0x87185069f2cb12c3:0xe78d657d5438d9a6, 0x8718...",https://www.google.com/maps/place//data

In [4]:
def analyze_dataframe(df, file_name, subset_column=None):
    # Obtén el nombre del archivo
    nombre_archivo = file_name

    # Obtenemos el número total de columnas y filas
    num_filas, num_columnas = df.shape

    # Obtenemos el número total de filas con valores nulos
    filas_con_nulos = df.isnull().any(axis=1).sum()

    # Obtenemos el número total de valores nulos por columna
    nulos_por_columna = df.isnull().sum()

    # Inicializamos la variable para las filas duplicadas
    filas_duplicadas = 0
    
    # Agregar el tipo de dato de cada columna
    tipos_de_datos = df.dtypes

    filas_duplicadas = df.duplicated(subset=subset_column).sum()

    # Agregar el tipo de dato de cada columna
    tipos_de_datos = df.dtypes

    # Construye el análisis del archivo actual
    analisis = f"Análisis del archivo: \"{nombre_archivo}\":  \
    \n- Primeras 5 filas del archivo\n {df.head(5)}\
    \n- Número total de columnas: {num_columnas}  \
    \n- Número total de filas: {num_filas}  \
    \n- Número total de filas duplicadas: {filas_duplicadas}  \
    \n- Número total de filas con valores nulos: {filas_con_nulos}  \
    \n- Número total de valores nulos y tipo de dato por columna:\n"
    
    for columna, nulos, tipo in zip(nulos_por_columna.index, nulos_por_columna.values, tipos_de_datos):
        analisis += f"{columna: <20} {nulos: <10} {tipo}\n"

    return analisis

In [8]:
# Especifica el nombre del bucket y el prefijo del directorio en GCS
bucket_name = 'project_yelp_parquet'
directory_prefix = 'reviews-estados/'

# Obtén una referencia al bucket
bucket = storage_client.get_bucket(bucket_name)

# Lista los archivos en el directorio especificado
blobs = bucket.list_blobs(prefix=directory_prefix)

# Filtra los archivos que terminan con '.csv'
archivos_parquet = [blob.name for blob in blobs if blob.name.endswith('.parquet')]

# Imprime la lista de archivos CSV
print(archivos_parquet)

['reviews-estados/review-Alabama.parquet', 'reviews-estados/review-Alaska.parquet', 'reviews-estados/review-Arizona.parquet', 'reviews-estados/review-Arkansas.parquet', 'reviews-estados/review-California.parquet', 'reviews-estados/review-Colorado.parquet', 'reviews-estados/review-Connecticut.parquet', 'reviews-estados/review-Delaware.parquet', 'reviews-estados/review-District_of_Columbia.parquet', 'reviews-estados/review-Florida.parquet', 'reviews-estados/review-Georgia.parquet', 'reviews-estados/review-Hawaii.parquet', 'reviews-estados/review-Idaho.parquet', 'reviews-estados/review-Illinois.parquet', 'reviews-estados/review-Indiana.parquet', 'reviews-estados/review-Iowa.parquet', 'reviews-estados/review-Kansas.parquet', 'reviews-estados/review-Kentucky.parquet', 'reviews-estados/review-Louisiana.parquet', 'reviews-estados/review-Maine.parquet', 'reviews-estados/review-Maryland.parquet', 'reviews-estados/review-Massachusetts.parquet', 'reviews-estados/review-Michigan.parquet', 'reviews

In [5]:
bucket_name = 'project_yelp_parquet'
file_path_user = 'reviews-estados/review-Alabama.parquet'
df_review_Alabama = read_parquet_from_gcs(bucket_name, file_path_user)

In [6]:
df_review_Alabama.sample(10)

,user_id,name,time,rating,text,pics,resp,gmap_id
587422,117177041892050452149,Jean Thompson,1615419835931,4,Clean. Just a little noise but enjoyable,None,None,0x886287df88a65bf3:0x730b8ab97cd14973
1455139,112647160901075795019,Melissa Grimes,1518007824897,5,None,None,None,0x887d49fbd3b508c1:0xacc6f2e3abf74575
743727,111229306974302186587,Dr. Thomas Bender,1533826951875,5,None,None,None,0x889a4d8e561bc15d:0xf543b0bd93e24a07
26561,107283017669394815758,Duce Tate,1511751623626,5,None,None,None,0x8889197af7e52653:0x7dd77af3794eb7d4
691268,113575492162011638492,April Henley,1578882339410,5,None,None,None,0x88620cec6eb43ef7:0x545bda964807f430
461980,113294639434145193541,Susan Glenn,1556710084111,3,None,None,None,0x889a0e3378b6338f:0x5729aa612f1b1fdb
232042,108530722004666041407,homesteader fifty w/ ricky & martha,1549121111697,5,None,None,None,0x8889222e1bf3cc35:0x1f20629b635975d2
1021083,106015993468339647334,Alabama Lucey,1565376388739,4,Good store and location. Store is clean with ...,None,"{'text': 'Thank you, Lucey! We take care to ke...",0x889a4b12ea2fff95:0x24b16280e5b0c30d
1681298,100477869172725127390,Terry Jones,1544545085065,5,None,None,None,0x88891c3b58c34e63:0xb954f8635cf9714a
1715330,106370560317027366314,Renee Hill,1571608027553,5,Great food an Great service! Will be back!,None,None,0x8889e67ed89cb081:0x23ca4646eeef6a0e


In [11]:
duplicates_review = df_review_Alabama.duplicated(['user_id', 'name', 'time', 'rating', 'text']).sum()
print(duplicates_review)

15948


In [12]:
nombre_archivo = "review-Alabama.parquet"
subset = ['user_id', 'name', 'time', 'rating', 'text']
analisis_resultado = analyze_dataframe(df_review_Alabama, nombre_archivo, subset)
print(analisis_resultado)

Análisis del archivo: "review-Alabama.parquet":      
- Primeras 5 filas del archivo
                  user_id                    name           time  rating  \
0  114043824230907811356           Kanisha Mixon  1597168272670       5   
1  116009008420407071921          Brandie Hodges  1609899039594       5   
2  106239928178034609776             Sharon King  1547235290843       4   
3  104970056115801744726         Veronica Pierce  1517709403534       5   
4  110587471700525107383  Whitney Waldon Collier  1535245718492       5   

                                                text  pics  resp  \
0  Very Personable staff! Beautiful and clean env...  None  None   
1                               Best clothing intown  None  None   
2                                               None  None  None   
3                                               None  None  None   
4                                               None  None  None   

                                 gmap_id  
0  0x886213

In [12]:
def process_parquet_files(bucket_name, directory_prefix, subset_column=None):
    # Crea una instancia de cliente de GCS
    storage_client = storage.Client()

    # Obtén una referencia al bucket
    bucket = storage_client.get_bucket(bucket_name)

    # Lista los archivos en el directorio especificado
    blobs = bucket.list_blobs(prefix=directory_prefix)

    # Filtra los archivos que terminan con '.parquet'
    archivos_parquet = [blob.name for blob in blobs if blob.name.endswith('.parquet')]

    # Lista para almacenar los resultados de análisis
    resultados_analisis = []

    # Procesa cada archivo Parquet
    for file_path in archivos_parquet:
        # Obtén el nombre del archivo sin el prefijo del directorio
        file_name = file_path.replace(directory_prefix, '')

        # Lee el archivo Parquet y obtén el DataFrame
        df = read_parquet_from_gcs(bucket_name, file_path)

        # Realiza el análisis del DataFrame y agrega el resultado a la lista
        analisis_resultado = analyze_dataframe(df, file_name, subset_column)
        resultados_analisis.append(analisis_resultado)

    return resultados_analisis

In [13]:
subset = ['user_id', 'name', 'time', 'rating', 'text']

resultados = process_parquet_files('project_yelp_parquet', 'reviews-estados/', subset)

In [15]:
print(resultados[0])

Análisis del archivo: "review-Alabama.parquet":      
- Primeras 5 filas del archivo
                  user_id                    name           time  rating  \
0  114043824230907811356           Kanisha Mixon  1597168272670       5   
1  116009008420407071921          Brandie Hodges  1609899039594       5   
2  106239928178034609776             Sharon King  1547235290843       4   
3  104970056115801744726         Veronica Pierce  1517709403534       5   
4  110587471700525107383  Whitney Waldon Collier  1535245718492       5   

                                                text  pics  resp  \
0  Very Personable staff! Beautiful and clean env...  None  None   
1                               Best clothing intown  None  None   
2                                               None  None  None   
3                                               None  None  None   
4                                               None  None  None   

                                 gmap_id  
0  0x886213

In [16]:
print(resultados[1])

Análisis del archivo: "review-Alaska.parquet":      
- Primeras 5 filas del archivo
                  user_id               name           time  rating  \
0  109129804842686204152         Nicki Gore  1566331951619       5   
1  113240926405758965692      Allen Ratliff  1504917982385       5   
2  113044837891141253355  Jonathan Tringali  1474765901185       4   
3  110329155147592031570             S Blad  1472858535682       4   
4  108989634908602011119     Daniel Formoso  1529649811341       5   

                                                text  pics  resp  \
0  We always stay here when in Valdez for silver ...  None  None   
1  Great campground for the price. Nice hot unlim...  None  None   
2  We tent camped here for 2 nights while explori...  None  None   
3  This place is just a few miles outside Valdez,...  None  None   
4  Probably the nicest and cleanest campground we...  None  None   

                                 gmap_id  
0  0x56b646ed2220b77f:0xd8975e316de80952  

In [17]:
print(resultados[2])

Análisis del archivo: "review-Arizona.parquet":      
- Primeras 5 filas del archivo
                  user_id          name           time  rating  \
0  118355965896157383742  Vicki Ullery  1628718071358       5   
1  112652295626320204307    Doc Kovacs  1609796129431       5   
2  108987901098798627143    Pedro Diaz  1594225668169       1   
3  106927662396085722919   Brandon Ray  1622676189509       5   
4  109053549375120469300    T. Bennett  1434829263041       5   

                                                text  pics  resp  \
0  Gwen was so informative and made me feel comfo...  None  None   
1  Dr. Huyhn is an ethical skilled dentist. Her s...  None  None   
2  Not sure if one star is fair but my situation ...  None  None   
3  Best dentist office I've ever had. Love everyt...  None  None   
4  I've been coming to Oasis dentistry for a whil...  None  None   

                                 gmap_id  
0  0x872b5dae3c5fcfa7:0xf0f8b286a2537821  
1  0x872b5dae3c5fcfa7:0xf0f8

In [18]:
print(resultados[3])

Análisis del archivo: "review-Arkansas.parquet":      
- Primeras 5 filas del archivo
                  user_id              name           time  rating  \
0  108317460579141970085      cayla layman  1630760310327       5   
1  102123313902332683437   thomas mcdaniel  1631038071292       1   
2  104067094003049131996       tony shelby  1627697592294       5   
3  107109337185185980940  james williamson  1624394171528       3   
4  114611735524979608325     Robert Hardin  1630615588505       5   

                                                text  \
0  Making West Memphis proud! Love this spot for ...   
1  Small burrito period high price. U dont get yo...   
2    I love the food and the staff are very friendly   
3  It was a New Restaurant and I never had there ...   
4                             Great food and service   

                                                pics  \
0  [{'url': ['https://lh5.googleusercontent.com/p...   
1                                               N

In [19]:
print(resultados[4])

Análisis del archivo: "review-California.parquet":      
- Primeras 5 filas del archivo
                  user_id           name           time  rating  \
0  108991152262655788985        Song Ro  1609909927056       5   
1  111290322219796215751    Rafa Robles  1612849648663       5   
2  112640357449611959087      David Han  1583643882296       4   
3  117440349723823658676    Anthony Kim  1551938216355       5   
4  100580770836123539210  Mario Marzouk  1494910901933       5   

                                               text  pics  resp  \
0                      Love there korean rice cake.  None  None   
1                                    Good very good  None  None   
2  They make Korean traditional food very properly.  None  None   
3                    Short ribs are very delicious.  None  None   
4      Great food and prices the portions are large  None  None   

                                 gmap_id  
0  0x80c2c778e3b73d33:0xbdc58662a4a97d49  
1  0x80c2c778e3b73d33:0xb

In [20]:
print(resultados[5])

Análisis del archivo: "review-Colorado.parquet":      
- Primeras 5 filas del archivo
                  user_id                            name           time  \
0  108990457316763116222                     mgshouldice  1484353876202   
1  107403966474786136203               Sabrina Holsinger  1484195642287   
2  112819818945043576491                       Tad Amore  1459554530689   
3  102591757433717355980  Victoria Elizabeth De La Torre  1483746469128   
4  104008941864455933539               Luis Garcia-Lopez  1542782383361   

   rating                                               text  pics  resp  \
0       3  Don't mix this up with their high school. It w...  None  None   
1       4  Teachers are great and you can tell the genual...  None  None   
2       5  Great school can't wait for my son to start ne...  None  None   
3       1                                               None  None  None   
4       5                                               None  None  None   

     

In [21]:
print(resultados[6])

Análisis del archivo: "review-Connecticut.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  114076516115211330289    Jeff Mccomish  1612059976820       5   
1  101520327069328234211      Outlaw PiRu  1544752859553       5   
2  111800206569311974834   Hector Acevedo  1564367962353       5   
3  112261838881776228573    Leah Williams  1610758109050       5   
4  108988845846950047344  Nick “Big Nick”  1570061475449       5   

                                                text  pics  resp  \
0                  Great friendly independent packie  None  None   
1  Great Atmosphere and Great Service. Fantastic ...  None  None   
2  Nice neighborhood store with great sandwiches ...  None  None   
3                                        Just trying  None  None   
4                                           Love it!  None  None   

                                 gmap_id  
0  0x89e64b04d274c9d9:0x84d833dcc10ae21d  
1  0x8

In [22]:
print(resultados[7])

Análisis del archivo: "review-Delaware.parquet":      
- Primeras 5 filas del archivo
                  user_id            name           time  rating  \
0  113797972931183350426  Heather Carper  1507258129698       5   
1  108911577609871083315  STACY CLAVETTE  1544802238870       5   
2  110147089361428367751       Zion Hood  1559123933643       5   
3  114714285192292470942     Diane Clark  1600736020990       5   
4  108986391413141739824         Eddie S  1554953350224       4   

                                                text  \
0  Lived here for 3 years and enjoyed it. Locatio...   
1  Nice complex and awesome staff.  Maintenance i...   
2  Good places for people to live my friend lives...   
3                                      great layout.   
4  Positives: Elevator, dog park and maintenance....   

                                                pics  resp  \
0  [{'url': ['https://lh5.googleusercontent.com/p...  None   
1                                               N

In [23]:
print(resultados[8])

Análisis del archivo: "review-District_of_Columbia.parquet":      
- Primeras 5 filas del archivo
                  user_id            name           time  rating  \
0  104256407771930872120       Petre Ene  1551027526726       4   
1  108642560086289718425  Keshava Mysore  1510920735052       4   
2  111842880135815090774         Carol G  1543249586598       2   
3  110299397397951052863              翟泉  1496597796978       4   
4  116447973760719145759       Kitae Yim  1540701399450       1   

                                                text  \
0  the food is really good, I had a great experie...   
1  High prices for not so big portions! But I thi...   
2  The gelato looked "old" and ended up having co...   
3  (Translated by Google) The fast food restauran...   
4  (Translated by Google) The food is barely edib...   

                                                pics  resp  \
0  [{'url': ['https://lh5.googleusercontent.com/p...  None   
1                                    

In [24]:
print(resultados[9])

Análisis del archivo: "review-Florida.parquet":      
- Primeras 5 filas del archivo
                  user_id              name           time  rating  \
0  101471856155148729010   Julie A. Gerber  1628003250740       1   
1  115477234789038326051  Martin Sheffield  1595031217005       5   
2  101805010244892834381      Brian Truett  1522924253567       5   
3  106344422881493743981          Tina Sun  1467907819586       1   
4  100875113069561776529      James Haynes  1480683415081       5   

                                                text  pics  \
0  Update: Their “reply” to my review amounted to...  None   
1  He's a knowledgeable doctor but the way he run...  None   
2  Best doctor I've ever had, I never wait to be ...  None   
3  I was told he is a good doctor. I was trying t...  None   
4  Takes the time to actually get to know his pat...  None   

                                                resp  \
0  {'text': 'Thank you for the feedback. We would...   
1  {'text': 'T

In [25]:
print(resultados[10])

Análisis del archivo: "review-Georgia.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  104227684932200174741     Brent Powell  1599604812269       5   
1  105572260232544349578      Evan Thomas  1502078169146       5   
2  103955436073218468595     Caleb Howell  1502571430852       5   
3  106974620002238817356  ashlee crawford  1494161075405       5   
4  104845669459415055814          T Paulk  1495139229501       5   

                                                text  pics  resp  \
0  Great hometown folk. Rob takes good care of al...  None  None   
1  Very friendly hometown pharmarcy. Always helpf...  None  None   
2                                               None  None  None   
3                                               None  None  None   
4                                               None  None  None   

                                 gmap_id  
0  0x88f16e41928ff687:0x883dad4fd048e8f8  
1  0x88f16

In [26]:
print(resultados[11])

Análisis del archivo: "review-Hawaii.parquet":      
- Primeras 5 filas del archivo
                  user_id               name           time  rating  \
0  113965417079576625433    manuel grimaldo  1591839903487       5   
1  116655819137293331166       Enrique Lara  1568059018979       5   
2  100834119994550070853  Gregory Donaldson  1594885588335       5   
3  103207214144482097315        Brian Baker  1575951131613       5   
4  108526171163172578599              Kam J  1573076723916       3   

                text  pics  resp                                gmap_id  
0  Great new upgrade  None  None  0x7c00159b5b1b1d25:0x8d2d85d4a758290e  
1               None  None  None  0x7c00159b5b1b1d25:0x8d2d85d4a758290e  
2               None  None  None  0x7c00159b5b1b1d25:0x8d2d85d4a758290e  
3               None  None  None  0x7c00159b5b1b1d25:0x8d2d85d4a758290e  
4               None  None  None  0x7c00159b5b1b1d25:0x8d2d85d4a758290e      
- Número total de columnas: 8      
- Número t

In [27]:
print(resultados[12])

Análisis del archivo: "review-Idaho.parquet":      
- Primeras 5 filas del archivo
                  user_id                                     name  \
0  107842099430937493658                            Jason Johnson   
1  103512058458520968675                            Michael Harry   
2  110297637817473583585                            Jolene Vaughn   
3  115057612098014688097                                Ben Voris   
4  104908091478986095236  Sunshine Window Cleaning Inc. Greg Reid   

            time  rating                                               text  \
0  1499810222323       3  Nice place, but not enough dog friendly areas ...   
1  1478021694645       5  Pheasant, ducks, geese, white tailed deer.\nSm...   
2  1533847989090       3  We took some horses up to ride, but there real...   
3  1563297497573       3  Crowded during pheasant season. If you’re not ...   
4  1543076221896       4  Beautiful spot for a hike or some hunting. Be ...   

                          

In [28]:
print(resultados[13])

Análisis del archivo: "review-Illinois.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  109442577269176737834    Jim LeBurkien  1571842572977       5   
1  100924480938771474648     Steve Muscat  1561679275695       5   
2  100813976190002245469  Fletcher Oliver  1572719516502       5   
3  110168499961967096526    Daniel Chavez  1572641473424       5   
4  108990598812734370097  David Ferdinand  1561483351910       4   

                                                text  pics  resp  \
0  Great selection of used and collectible toys. ...  None  None   
1                           Great visit as usual. 👍😊  None  None   
2                                               None  None  None   
3                                               None  None  None   
4                                               None  None  None   

                                 gmap_id  
0  0x880f0f1b32155555:0x8347e0c971acb955  
1  0x880f

In [29]:
print(resultados[14])

Análisis del archivo: "review-Indiana.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  111423711419019424734   Larry Sandefur  1615400502414       5   
1  113596625824447012651       John Soard  1597280425295       5   
2  100821699956425809518  Michael Johnson  1596678908723       5   
3  106063043906772778894    Lauren Moretz  1606588867988       1   
4  107227282238314902977    Tracy Workman  1606677082659       5   

                                                text  pics  \
0  Majestic security was the best job I ever had ...  None   
1  I have used this company for a number of years...  None   
2  I am an employee with Majestic Security, and h...  None   
3  My husband has worked for this company for a f...  None   
4  I'm 55 and have been here with Magestic over 6...  None   

                                                resp  \
0  {'text': 'Thanks for the review Larry, and for...   
1  {'text': 'Thanks 

In [30]:
print(resultados[15])

Análisis del archivo: "review-Iowa.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  115499682928803397674  Nicole Williams  1528935574441       5   
1  110072671534171333918        Dmitriy N  1575832576112       5   
2  117021476131280107146      Sue Dickman  1465139905809       5   
3  109003155465609632997    Ashley Nguyen  1565271263810       5   
4  113278552835557781367     Candy Hester  1450564402784       5   

                                                text  pics  \
0  I have found my new favorite nail shop. I love...  None   
1  This place is awesome. Got my pedicure there a...  None   
2  I love going to Polished Nails. Everyone is pr...  None   
3                  Nice place to get a complete set.  None   
4  Great manicure from someone who really cares f...  None   

                                                resp  \
0        {'text': 'Thanks', 'time': 1528950923185.0}   
1  {'text': 'Thanks , D

In [31]:
print(resultados[16])

Análisis del archivo: "review-Kansas.parquet":      
- Primeras 5 filas del archivo
                  user_id               name           time  rating  \
0  110174851715143039572    Hannah Johnston  1603469475914       4   
1  109469133033675413034  Flint Blankenship  1628280826510       4   
2  101132804334372412402      Sheryl Fields  1511648800146       2   
3  114573410811170173535       Robert Gragg  1572660533736       4   
4  117268927797389286107           Jan Swan  1603471293093       4   

                                                text  pics  \
0  Really cute store. All the staff are lovely. T...  None   
1       Well......  The place is a 5-Star..... BUT..  None   
2  Normally we really like stopping in and shoppi...  None   
3  Nice selection of antiques collectables and ho...  None   
4                                         Nice stuff  None   

                                                resp  \
0                                               None   
1        

In [32]:
print(resultados[17])

Análisis del archivo: "review-Kentucky.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  117631854574648614468  Fatcat Paulanne  1612545195499       5   
1  108986538807550589214     Scott Vatter  1531435973696       4   
2  108762923036268899825       Cait Lewis  1505925988144       2   
3  104536045402580055063    Kristen Casey  1558918298929       5   
4  108980494708640256888     Cheryl Basil  1507352415093       5   

                                                text  pics  resp  \
0  Very professional.  Returned phone calls promp...  None  None   
1                                       Very helpful  None  None   
2                                               None  None  None   
3                                               None  None  None   
4                                               None  None  None   

                                 gmap_id  
0  0x886851e247062e25:0xd455b2d55eda354a  
1  0x8868

In [33]:
print(resultados[18])

Análisis del archivo: "review-Louisiana.parquet":      
- Primeras 5 filas del archivo
                  user_id                 name           time  rating  \
0  110117661441630099731        Taria Jackson  1583803141563       5   
1  104437134567604799205         Jodi Kociaba  1603610106892       5   
2  108991048262705869218            Beejal S.  1527773942699       5   
3  107509536819719046474          Lovely 2021  1578657721674       5   
4  114498177305142292489  Technician Ash Mohd  1580787625014       5   

                                                text  pics  \
0  Reasonably priced, process was easy and fast! ...  None   
1      Was quick, easy and fun to use the key kiosk.  None   
2  Easy to use, fairly fast.  $3.49 for copy of b...  None   
3                                               None  None   
4                                               None  None   

                                                resp  \
0  {'text': 'Taria , thank you so much for taking.

In [34]:
print(resultados[19])

Análisis del archivo: "review-Maine.parquet":      
- Primeras 5 filas del archivo
                  user_id           name           time  rating  \
0  113439885720143786522  Steve Holland  1528823744253       4   
1  107421546746170771489   Matthew Hall  1500129522343       2   
2  103835206402034953214   Laura Oliver  1534025979137       1   
3  113493171757583101688      Phil Rose  1495584198169       2   
4  108990334175672695739       Sam I Am  1436912786006       4   

                                                text  pics  resp  \
0  My friend and I played here today. played all ...  None  None   
1  It literally pains me to give such a low revie...  None  None   
2  DO NOT GO HERE!!! Seriously!! This was the mos...  None  None   
3  Not sure what happened but it's not what it us...  None  None   
4  Very nice course it's awesome that 27 holes al...  None  None   

                                 gmap_id  
0  0x4cb27993a61ab3e1:0xa6c6508fe8842ff0  
1  0x4cb27993a61ab3e1:0x

In [35]:
print(resultados[20])

Análisis del archivo: "review-Maryland.parquet":      
- Primeras 5 filas del archivo
                  user_id              name           time  rating  \
0  116210082545550472083           Brian A  1528719383671       5   
1  108990491488903625674          Marsha J  1556106162554       5   
2  117908754119590740399              Noel  1531995775977       5   
3  117447665198230000362  Sheila Dickerson  1533222175613       1   
4  114517700968499580611    Carlos Alvarez  1489337608868       5   

                                                text  pics  \
0  Every time I have been to this place I am in a...  None   
1  Good pharmacy and fast service, majority of th...  None   
2                              Awesome service here!  None   
3                  I never received my prescription.  None   
4                                          very fast  None   

                                                resp  \
0                                               None   
1  {'text': '

In [36]:
print(resultados[21])

Análisis del archivo: "review-Massachusetts.parquet":      
- Primeras 5 filas del archivo
                  user_id              name           time  rating  \
0  118033960754031370625     Donna Carlson  1619732714443       5   
1  110167250775171856468   michael simmons  1615400031636       5   
2  112213606714604522444       Paul Murray  1593992657241       5   
3  108174492472565092745  Elba De Oliveira  1582074606394       5   
4  108990132658983842962      Sherri Mayne  1572962045881       5   

                                                text  pics  resp  \
0        People run it make sure you are comfortable  None  None   
1                      When you need a helping hand.  None  None   
2  Very Helpful and understanding people definite...  None  None   
3  Very good place nice things but on Saturday th...  None  None   
4                                      Awesome place  None  None   

                                 gmap_id  
0  0x89e4a103dd188585:0xfadc47938337579f 

In [37]:
print(resultados[22])

Análisis del archivo: "review-Michigan.parquet":      
- Primeras 5 filas del archivo
                  user_id           name           time  rating  \
0  114997453234903127706        Deb Mae  1611524448905       5   
1  104617682891646375505   sherry stone  1574805324619       1   
2  106438016631857553727  Shannon Kropp  1455651168542       5   
3  105314300453777917358        M A T T  1595369812147       5   
4  107590881528799662566   Not the Mama  1593635254254       5   

                                                text  pics  \
0  I have been with Sara as my Agent for many yea...  None   
1  I went to this allstate because I needed a new...  None   
2  Everyone at this office is great!  They make m...  None   
3                                      Great service  None   
4                                           Friendly  None   

                                                resp  \
0  {'text': 'Deb,  Thank you so much for saying t...   
1                              

In [38]:
print(resultados[23])

Análisis del archivo: "review-Minnesota.parquet":      
- Primeras 5 filas del archivo
                  user_id            name           time  rating  \
0  112913127230032897502       king 8488  1600976328270       5   
1  116255773650541805963  Steve Thornton  1544372192905       5   
2  101904989862576038089   Jessica Perry  1464405464526       5   
3  104261549918960965051   Frank Beissel  1537928043548       4   
4  102305414133657656744     Ryan Kwyzla  1538256770007       5   

                                                text  pics  \
0  Super helpful local business they are there fo...  None   
1  I was renting at a popular nation wide chain a...  None   
2  Hands down the best place ever! Ran by super n...  None   
3  Edit.  I was wrong. Wrong place.\n\nThey seem ...  None   
4  Super friendly and helpful staff. Location is ...  None   

                                                resp  \
0  {'text': 'Thank you for the kind words. We're ...   
1  {'text': 'Thank you S

In [39]:
print(resultados[24])

Análisis del archivo: "review-Mississippi.parquet":      
- Primeras 5 filas del archivo
                  user_id            name           time  rating  \
0  109818626919525938711  Patti Strother  1598023328629       1   
1  108988793398570508962  Hilda Forister  1492212551217       5   
2  109244745778779889928            J TH  1530800061632       5   
3  114786386029494847683    Tim Ashcraft  1447764412977       3   
4  108370079228885751617    DAVE MARABLE  1484427496391       5   

                                                text  pics  \
0  Don’t Use your DEBIT CARD at this location! Sk...  None   
1  Every time I have gone it's either for gas or ...  None   
2  There's a young black male who works here that...  None   
3  No. This is not always the best gas price arou...  None   
4  Staff and service are great!!! Chicken combo i...  None   

                                                resp  \
0                                               None   
1                     

In [40]:
print(resultados[25])

Análisis del archivo: "review-Missouri.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  108988165773390633965     Tracy Reeder  1575932557871       1   
1  100477628820189377837    Brett Stevens  1554489896899       5   
2  118170132622286925785  steven williams  1498137347552       5   
3  108527402622595658562                b  1511496601769       5   
4  103401154512547790816      Bekkah said  1505363292500       5   

                                                text  pics  resp  \
0  This company seems like a great place to work ...  None  None   
1                                               None  None  None   
2                                               None  None  None   
3                                               None  None  None   
4                                               None  None  None   

                                 gmap_id  
0  0x87c87e473635feb5:0x6de42f63fb4ad19b  
1  0x87c8

In [41]:
print(resultados[26])

Análisis del archivo: "review-Montana.parquet":      
- Primeras 5 filas del archivo
                  user_id                name           time  rating  \
0  114771463617399560212  Shifts and giggles  1618975510300       5   
1  100577710152975396781        Craig Thomas  1609450506236       5   
2  105204186482577003750       Lance Broshar  1595447399474       5   
3  110521592956426830493        Inicorn Refs  1569377416664       5   
4  115857374920568014593                Chad  1607551230333       5   

                                                text  pics  resp  \
0  Good people that know thier stuff excellent cu...  None  None   
1             Fixed my saw bout 10min.\n\n1st class.  None  None   
2  Excellent service and products. Fair and reaso...  None  None   
3  One day they may start accepting credit cards!...  None  None   
4                            Great place good people  None  None   

                                 gmap_id  
0  0x535e9ec3ce050deb:0x2169887374c

In [42]:
print(resultados[27])

Análisis del archivo: "review-Nebraska.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  108846811029680511029       Dan Grundy  1608331653332       5   
1  116191401875892866913  dominick vacura  1627079809988       5   
2  108985626010316415557      Cody Albers  1533832546055       4   
3  116926779203304046512    Markus Farmer  1559062491880       5   
4  114721354930264722920     james spevak  1526077661056       5   

                                                text  pics  resp  \
0  For give me for not catching the managers name...  None  None   
1                                   I've got shades!  None  None   
2  Went here to see if I could get my glasses adj...  None  None   
3  Very helpful with questions. More Versace plea...  None  None   
4  Pretty good place to look for sunglasses. But ...  None  None   

                                gmap_id  
0  0x8793f2b4d692a433:0xc4d08ac32475203  
1  0x8793f2

In [43]:
print(resultados[28])

Análisis del archivo: "review-Nevada.parquet":      
- Primeras 5 filas del archivo
                  user_id          name           time  rating  \
0  102843170183026312173        CDL CO  1622740504476       5   
1  114954392137907747077     So_ Blind  1573159413820       1   
2  116523433330089119119   Greg Hansen  1515251163362       3   
3  103304053791281129156  Brett Powell  1506206762095       3   
4  114162416344450979054        M Aris  1468298771892       3   

                                                text  \
0  No frills smaller coffee counter in the corner...   
1  Save yourself the frustration and just ignore ...   
2  I passed two Starbucks in order to get to this...   
3  It does the job and is my preference over the ...   
4                                               None   

                                                pics  resp  \
0  [{'url': ['https://lh5.googleusercontent.com/p...  None   
1                                               None  None   
2

In [44]:
print(resultados[29])

Análisis del archivo: "review-New_Hampshire.parquet":      
- Primeras 5 filas del archivo
                  user_id               name           time  rating  \
0  107325861918074131473  Jeffrey J Howland  1621473866341       4   
1  113012300176598630052    Nathaniel Houle  1604724170325       5   
2  105839925626034653337     Shannon Thomas  1585573760111       5   
3  106651075189339207621         Lora Haase  1593363624213       5   
4  113037354732826983651       David Blouin  1593808080254       4   

                                                text  pics  resp  \
0  Like many campgrounds dust is a problem... but...  None  None   
1  Nice clean and welcoming campground.  Love the...  None  None   
2  ❤Super friendly people and very well maintaine...  None  None   
3  Great time. Owners are awesome.  Had a great w...  None  None   
4  Seems like a nice family owned campground the ...  None  None   

                                 gmap_id  
0  0x89e2936fffffffff:0x3fce71a3d83

In [45]:
print(resultados[30])

Análisis del archivo: "review-New_Jersey.parquet":      
- Primeras 5 filas del archivo
                  user_id               name           time  rating  \
0  101190277000060800745       Monica Zobbi  1614634731992       5   
1  117269913257051471311  Vitaliia Stefurak  1572974792541       5   
2  115226987362661223566       Maria Guzman  1578525387607       5   
3  116182780268968248990             Jo Rad  1602103279588       5   
4  101071187098115538507     Kimberly Hults  1593034107348       5   

                                                text  pics  \
0  Viktoriya is always very professional and atte...  None   
1  If I could leave them 10 starts I would , unbe...  None   
2  Laura took her time to know my hair and what s...  None   
3  Wow, what an experience! Haircut by Victoria! ...  None   
4  Love this salon! The great experience starts o...  None   

                                                resp  \
0                                               None   
1    

In [46]:
print(resultados[31])

Análisis del archivo: "review-New_Mexico.parquet":      
- Primeras 5 filas del archivo
                  user_id           name           time  rating  \
0  109015130651419577845    Tania Solis  1607374166577       5   
1  114641989724535457798  James Watters  1558380690011       5   
2  118173316599145514941       Jesse G.  1542751368749       1   
3  115908859152987365493           TR S  1486093061400       4   
4  108981200384503827560   Mannie Myers  1625862764044       4   

                                                text  pics  \
0  I was helped by Rachel and let me tell you! Sh...  None   
1  Colton was fantastic to work with and our shir...  None   
2  I've been a returning customer for sometime an...  None   
3                  Flighty service. Great selection.  None   
4                                               None  None   

                                                resp  \
0  {'text': 'Thank you Tania! We'll pass the mess...   
1  {'text': 'Thank you James!

In [47]:
print(resultados[32])

Análisis del archivo: "review-New_York.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  113722104692308235141   Alvin Martinez  1603494795361       5   
1  107293441492109320298  Johnnie Jackson  1620157037403       1   
2  100378585801819400296     Manie Blazer  1597431662039       5   
3  114998161153019826512   Fashion Fiinds  1543773862044       5   
4  117178185728422297915   Andres Rieloff  1597279097718       1   

                                                text  \
0  I'm late to posting this but this store especi...   
1  Very dissatisfied I did not get my phone the p...   
2  Excellent very well done with professional car...   
3  Basing my review strictly on the service I rec...   
4  Bad! Disorganized. I'm being totally honest. I...   

                                                pics  \
0  [{'url': ['https://lh5.googleusercontent.com/p...   
1                                               None   

In [48]:
print(resultados[33])

Análisis del archivo: "review-North_Carolina.parquet":      
- Primeras 5 filas del archivo
                  user_id              name           time  rating  \
0  117341286276225129230     Vicki Pittman  1624283263383       5   
1  115729692379822395024     Harley Reaves  1571655539426       5   
2  111547769181082215438    JESSICA ALFORD  1590856630354       5   
3  109110552668209780158  Tyler Piotrovsky  1577651422743       5   
4  112039109323514983001    Frank Bissette  1590252300386       5   

                                                text  \
0  Son gets hair cut. Kat is his stylist/barber. ...   
1  Really great group of girls! If you have the e...   
2  Kat is amazing! My son loves the way she gives...   
3  Samantha gave me a perfect cut for me, though ...   
4       As always Kat is the greatest stylist ever!!   

                                                pics  resp  \
0                                               None  None   
1                              

In [49]:
print(resultados[34])

Análisis del archivo: "review-North_Dakota.parquet":      
- Primeras 5 filas del archivo
                  user_id               name           time  rating  \
0  102470746484263305736       Bryan Miller  1626317971375       5   
1  102212743628894360920        Kurt Franke  1612038279541       5   
2  112520415779711157106       Amber Moller  1623440198128       5   
3  113398177034208080372          Ian Smith  1619722402903       5   
4  116014249099302040560  Lilslayerplaez .W  1594748268627       5   

                                                text  pics  resp  \
0  Fast quick service. I go here almost all the t...  None  None   
1  Mike and crew do a great job.  Dropped off veh...  None  None   
2                           Amazing love this place!  None  None   
3                            Fair prices speedy work  None  None   
4  Always in & out.  Super professional and conve...  None  None   

                                 gmap_id  
0  0x52d0147640d529bd:0xec9407dc9201

In [50]:
print(resultados[35])

Análisis del archivo: "review-Ohio.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  114733505901720471519  Justin Calabris  1584232912307       1   
1  115546115125505203319    Benjamin Cope  1487429476527       5   
2  109114609771331992238    Jason Hammond  1629077164606       5   
3  115969333473786857055           a Null  1485737926073       2   
4  113888870632401106244         Justin F  1499085386882       4   

                                    text  pics  \
0                 I would not recommend.  None   
1  Quality communication and workmanship  None   
2                                   None  None   
3                                   None  None   
4                                   None  None   

                                                resp  \
0  {'text': 'Justin, I am sorry to hear that you ...   
1                                               None   
2                                      

In [51]:
print(resultados[36])

Análisis del archivo: "review-Oklahoma.parquet":      
- Primeras 5 filas del archivo
                  user_id            name           time  rating  \
0  108990106903261944178  Dana Schreiber  1630078007996       5   
1  113811268211478361908     Julie Combs  1612485567960       3   
2  102455850197465168450        Regi Glo  1598387347506       5   
3  103762530008535859564    Jaimee Steed  1593476994427       5   
4  116430839116161622337   Kaylee Miller  1575156563539       1   

                                                text  pics  \
0  Very knowledgeable, you have a question they k...  None   
1  Well they had what I needed but the only  Budt...  None   
2  Great place to shop... very helpful and they a...  None   
3  Sara is fabulous and super sweet! Lots of choi...  None   
4  This place always advertises good deals to get...  None   

                                                resp  \
0                                               None   
1                        

In [52]:
print(resultados[37])

Análisis del archivo: "review-Oregon.parquet":      
- Primeras 5 filas del archivo
                  user_id               name           time  rating  \
0  116238557567455956213          Mike Alan  1607013385211       5   
1  116988773242398246268   Jennifer Zumwalt  1607571121183       5   
2  118003928746583471938  Courtney Saunders  1582762237754       5   
3  118003928746583471938  Courtney Saunders  1582762237754       5   
4  103097287486867336142       Becky proulx  1584064731306       5   

                                                text  pics  \
0  Every staff member in this office is absolutel...  None   
1  They are amazing. Always so kinda and helpful....  None   
2  With three kids and myself, our family has bee...  None   
3  With three kids and myself, our family has bee...  None   
4  Three of our children have received their brac...  None   

                                                resp  \
0                                               None   
1        

In [53]:
print(resultados[38])

Análisis del archivo: "review-Pennsylvania.parquet":      
- Primeras 5 filas del archivo
                  user_id               name           time  rating  \
0  104944742559079759337    Jaron Whitfield  1517731762839       5   
1  117609702832980320437  Jonathan McCarthy  1476276291163       5   
2  110563242018426632956          Rocky Kev  1338826945578       5   
3  112895973505401399406        Josep Valls  1363286110554       5   
4  110619674885963828884     Timaree Schmit  1548798329760       5   

                                                text  pics  resp  \
0  Joe is quite unique of his line of work, he as...  None  None   
1  For such a small place their impact on my life...  None  None   
2  I usually give them a call before I stop by to...  None  None   
3  My bike had been sitting outdoors for a good w...  None  None   
4  Always an easy experience. Service is knowledg...  None  None   

                                 gmap_id  
0  0x89c6c63c8cd87141:0x54d0d283872e

In [54]:
print(resultados[39])

Análisis del archivo: "review-Rhode_Island.parquet":      
- Primeras 5 filas del archivo
                  user_id                  name           time  rating  \
0  107906906955290728264           Rubia Bella  1601590520320       5   
1  116735009919665557057       angel olivencia  1602983693555       4   
2  105746891382332508275       Daniel Reggiani  1603064156784       5   
3  116728227707437984696  Joseph “Joe” Solomon  1600710962505       5   
4  111306322994544036169        Julio Robinson  1601408236165       5   

                                  text  pics  \
0  Very good! Will eat here again! ☺️😍  None   
1           Not bad place... good food  None   
2                     Really good food  None   
3                Great people and food  None   
4                                 None  None   

                                                resp  \
0  {'text': 'Hope to see you back again thanks fo...   
1                                               None   
2            

In [55]:
print(resultados[40])

Análisis del archivo: "review-South_Carolina.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  108990006967932198283     Pamela Poole  1611327187871       4   
1  115786965178279773121  Cleveland Grant  1520462369713       5   
2  100954254888344692120       Sean Smith  1492902201253       5   
3  117716818447393496651  ThunderStroke91  1509505441582       5   
4  103062008704992053963    Joshua Tongue  1578438229754       5   

                                                text  \
0  They were very informative  about  the product...   
1  Great Gun Shop!!! Best local shop you will fin...   
2  Met Jr and Cathy today at the gun show in Gree...   
3  Big variety, they've got what you need. They a...   
4                                               None   

                                                pics  resp  \
0                                               None  None   
1  [{'url': ['https://lh5.googleuserc

In [56]:
print(resultados[41])

Análisis del archivo: "review-South_Dakota.parquet":      
- Primeras 5 filas del archivo
                  user_id               name           time  rating  \
0  103563353519118158246          Peri Gray  1516122675780       5   
1  101824980797027243148        Suzy Berndt  1532922350314       5   
2  108711640480272772398  Rosemary Red Legs  1530969093932       5   
3  101852294221648460926         Brown Wolf  1537085635922       2   
4  108987444312280650947      C J Blue Coat  1474922375491       5   

                                    text  pics  resp  \
0  Great place to care for our children.  None  None   
1                    Th sw y are so nice  None  None   
2                  Went with my daughter  None  None   
3                                   None  None  None   
4                                   None  None  None   

                                 gmap_id  
0  0x532af45db8f30779:0xd9be9359f1e56178  
1  0x532af45db8f30779:0xd9be9359f1e56178  
2  0x532af45db8f30779:

In [57]:
print(resultados[42])

Análisis del archivo: "review-Tennessee.parquet":      
- Primeras 5 filas del archivo
                  user_id            name           time  rating  \
0  118288383279870497662     Ally Medina  1567349037857       1   
1  113084676538676678822    Nikki Thomas  1476897445568       5   
2  100686891229653117464   Michael Wahba  1574100089433       5   
3  100547995478573859128     DaiJa Ricks  1535731263264       5   
4  105957410396834148372  Lydia Thompson  1554665980076       5   

                                                text  pics  \
0  I have been taking my daughter here since 2015...  None   
1  We switched from Dr.Henschel at Tennessee Pedi...  None   
2  The doctors and nurses and all staff are reall...  None   
3  My children have been patients at this office ...  None   
4  Dr. Haselton is great, my boys have seen her f...  None   

                                                resp  \
0  {'text': 'All of the providers of Nurture Pedi...   
1                       

In [58]:
print(resultados[43])

Análisis del archivo: "review-Texas.parquet":      
- Primeras 5 filas del archivo
                  user_id              name           time  rating  \
0  110545299078970317447    Kimberly Feger  1625431734826       5   
1  103619788097506115343     Briana Streit  1568336283913       2   
2  101916663109091637233   Sylvia Caudillo  1519873586872       1   
3  117870898304582507607     Ginger Kinyon  1571638384637       1   
4  110397346115416712442  Angeles Arellano  1546568877087       1   

                                                text  pics  resp  \
0  The pharmacist, Erin, is phenomenal. She was s...  None  None   
1  I gave them 2 stars because they offer prescri...  None  None   
2  If I could put minus stars I would. This has t...  None  None   
3                     Please fix your restroom doors  None  None   
4  This pharmacy Walmart dose not work not come a...  None  None   

                                 gmap_id  
0  0x864c3998b8d8dc83:0x57ffabe1e2322320  
1  0x8

In [59]:
print(resultados[44])

Análisis del archivo: "review-Utah.parquet":      
- Primeras 5 filas del archivo
                  user_id              name           time  rating  \
0  111182595077674366891        Liz W Poch  1627085008811       5   
1  117689170846448249054  Mario D'Ambrosio  1626437133578       1   
2  115834685654999484011   Kathleen Hambly  1585165423975       5   
3  112679611213731715979     Crystal Olsen  1591629458998       1   
4  108599429350262421326      Sarah Jensen  1510264647735       5   

                                                text  pics  \
0  Paige is the best! Everyone at this branch is ...  None   
1  I have updated my systems on two titles for ca...  None   
2  Simone is the best. She helped me to defer my ...  None   
3  Ive called several times and there customer se...  None   
4  My fiance and I had an awesome stress-free exp...  None   

                                                resp  \
0                                               None   
1                

In [60]:
print(resultados[45])

Análisis del archivo: "review-Vermont.parquet":      
- Primeras 5 filas del archivo
                  user_id             name           time  rating  \
0  118026874392842649478    rebecca kerns  1620085852324       5   
1  101532740754036204131    Peter DeForge  1580309946474       5   
2  115404122636203550540    Chad Goulette  1605195974445       5   
3  104789336434407408181  Mark LaFountain  1593005848256       5   
4  108980665975608069965           Jeff R  1582059996120       5   

                                                text  pics  \
0      Always done right from wood stove to screens!  None   
1  A great company to work with.  Their sales and...  None   
2  Great place to do business with staff was grea...  None   
3  Awesome Customer service, quick response, and ...  None   
4  If you need a top quality job, by a group of p...  None   

                                                resp  \
0  {'text': 'Good Evening, Rebecca! Thanks SO muc...   
1  {'text': 'Good Af

In [61]:
print(resultados[46])

Análisis del archivo: "review-Virginia.parquet":      
- Primeras 5 filas del archivo
                  user_id            name           time  rating  \
0  111603605060980047815      Jessie Lee  1581455645898       5   
1  106278681944354596566  Brittany Janis  1612044004790       2   
2  107562899595101737871    Brandy Monge  1580144842073       5   
3  100074131729110835897     Lisa Adkins  1612916442287       4   
4  101505548082630172302     Bill Bercaw  1549752949680       2   

                                                text  pics  \
0                     I loved Rita!! She was awesome  None   
1                         Expensive to do taxes here  None   
2    She was very helpful and made sure I understood  None   
3                                      Tracey Spadey  None   
4  I waited 1.5 hours for my Appointment because ...  None   

                                                resp  \
0                                               None   
1  {'text': 'Thank you fo

In [62]:
print(resultados[47])

Análisis del archivo: "review-Washington.parquet":      
- Primeras 5 filas del archivo
                  user_id              name           time  rating  \
0  103093043835388050629     Casper Steele  1626907411534       2   
1  111014066796803341223         Judy Maes  1613028426989       5   
2  111724423355988809570  Daniel Hernandez  1536710665852       5   
3  115331094085411087904         Lumi Nosa  1484359806540       5   
4  106906383883851362879    James Conright  1500643741079       1   

                                                text  pics  resp  \
0  Drivers say security has bathroom key. Securit...  None  None   
1  It's a CTRAN transit center with schedules to ...  None  None   
2  Neat little Transit Center with lots of schedu...  None  None   
3                              Very helpful.  Thanks  None  None   
4                             They don't sell tacos.  None  None   

                                 gmap_id  
0  0x5495ae7d3bf7d097:0xbcbc06152a3ccebc  
1

In [63]:
print(resultados[48])

Análisis del archivo: "review-West_Virginia.parquet":      
- Primeras 5 filas del archivo
                  user_id          name           time  rating  \
0  104649161250561778800   Pam Holcomb  1557935316444       4   
1  100934414726750280374  Cheryl Norga  1532897953993       5   
2  109662770281820282865   Jason Lough  1551725058851       3   
3  109466936358607031918    Chris Cook  1574728085154       5   
4  102864727386608433454   Dave Graley  1539855732886       4   

                                                text  pics  resp  \
0  Nice personnel and atmosphere and have done bu...  None  None   
1  Maggie is awesome..very sweet & helpful.\nLadi...  None  None   
2   Little on the expensive side but friendly people  None  None   
3                     Too bad this place is closing.  None  None   
4                                               None  None  None   

                                 gmap_id  
0  0x884955486c37d473:0x1b1f0907f74deee5  
1  0x884955486c37d473:

In [64]:
print(resultados[49])

Análisis del archivo: "review-Wisconsin.parquet":      
- Primeras 5 filas del archivo
                  user_id               name           time  rating  \
0  105449565300564768957      Kadence Leigh  1592846644902       3   
1  108990867128544289815     Jeremy Schafer  1534088176466       1   
2  101144372502144488423      Kaden Lindner  1525430562021       1   
3  103419705898581075829            J ellen  1594403566473       1   
4  103054337970583178080  Stephanie Johnson  1575431324657       3   

                                                text  pics  resp  \
0  Fun although a lot of breaks witch sucks and t...  None  None   
1  Management sucks they don't watch the  kids ha...  None  None   
2  Horrible water so groups I saw a full diper on...  None  None   
3                                               None  None  None   
4                                               None  None  None   

                                 gmap_id  
0  0x87fd108d3f915a5f:0x2b7132092901f4d

In [65]:
print(resultados[50])

Análisis del archivo: "review-Wyoming.parquet":      
- Primeras 5 filas del archivo
                  user_id            name           time  rating  \
0  111435346941066248277     Jenn Thomas  1602893531994       5   
1  103613723635264236030  Michelle Barta  1575991509006       5   
2  113556685435236755030  Bethany Baldes  1626981484302       5   
3  102235352482458236558    nichole vogt  1583381337082       5   
4  114021734869233847378   Kristi Friday  1571112557173       4   

                                                text  pics  \
0  When knowledge is key and kindness matters, Ni...  None   
1  The entire team is outstanding! They are profe...  None   
2                                               None  None   
3                                               None  None   
4                                               None  None   

                                                resp  \
0                                               None   
1  {'text': 'We are lucky 

In [67]:
bucket_name = 'project_yelp_parquet'
ruta_archivo_gcs = 'resultados_analisis_reviews_estados.txt'
ruta_gcs = f'gs://{bucket_name}/{ruta_archivo_gcs}'
# Abre el archivo en modo de escritura en GCS
with storage_client.bucket(bucket_name).blob(ruta_archivo_gcs).open('w') as archivo:
    # Escribe cada resultado en una nueva línea
    for resultado in resultados:
        archivo.write(f"{resultado}\n")

print(f"Resultados guardados en {ruta_gcs}")

Resultados guardados en gs://project_yelp_parquet/resultados_analisis_reviews_estados.txt


In [17]:
def useful_rows(df, file_name, subset_column=None):
    # Obtén el nombre del archivo
    nombre_archivo = file_name

    # Eliminar filas duplicadas
    df = df.drop_duplicates(subset=subset_column)

    # Eliminar filas en las que todos los valores del subset son nulos
    if subset_column is not None:
        df = df.dropna(subset=subset_column, how='all')

    # Obtener el número de filas después de la limpieza
    num_filas = df.shape[0]

    # Construir el diccionario con la información solicitada
    result_dict = {nombre_archivo: int(num_filas)}

    return result_dict

In [18]:
nombre_archivo = "review-Alabama.parquet"
subset = ['user_id', 'name', 'time', 'rating', 'text']
analisis_resultado = useful_rows(df_review_Alabama, nombre_archivo, subset)
print(analisis_resultado)

{'review-Alabama.parquet': 1784052}


In [19]:
def useful_rows_parquet_files(bucket_name, directory_prefix, subset_column=None):
    # Crea una instancia de cliente de GCS
    storage_client = storage.Client()

    # Obtén una referencia al bucket
    bucket = storage_client.get_bucket(bucket_name)

    # Lista los archivos en el directorio especificado
    blobs = bucket.list_blobs(prefix=directory_prefix)

    # Filtra los archivos que terminan con '.parquet'
    archivos_parquet = [blob.name for blob in blobs if blob.name.endswith('.parquet')]

    # Lista para almacenar los resultados de análisis
    resultados_analisis = []

    # Procesa cada archivo Parquet
    for file_path in archivos_parquet:
        # Obtén el nombre del archivo sin el prefijo del directorio
        file_name = file_path.replace(directory_prefix, '')

        # Lee el archivo Parquet y obtén el DataFrame
        df = read_parquet_from_gcs(bucket_name, file_path)

        # Realiza el análisis del DataFrame y agrega el resultado a la lista
        analisis_resultado = useful_rows(df, file_name, subset_column)
        resultados_analisis.append(analisis_resultado)

    return resultados_analisis

In [20]:
subset = ['user_id', 'name', 'time', 'rating', 'text']

rows_to_analyze = useful_rows_parquet_files('project_yelp_parquet', 'reviews-estados/', subset)

In [21]:
rows_to_analyze

[{'review-Alabama.parquet': 1784052},
 {'review-Alaska.parquet': 516145},
 {'review-Arizona.parquet': 1910655},
 {'review-Arkansas.parquet': 2374297},
 {'review-California.parquet': 2623809},
 {'review-Colorado.parquet': 2362828},
 {'review-Connecticut.parquet': 2645648},
 {'review-Delaware.parquet': 891745},
 {'review-District_of_Columbia.parquet': 558178},
 {'review-Florida.parquet': 2729661},
 {'review-Georgia.parquet': 1894568},
 {'review-Hawaii.parquet': 1484935},
 {'review-Idaho.parquet': 2057505},
 {'review-Illinois.parquet': 2052916},
 {'review-Indiana.parquet': 2219803},
 {'review-Iowa.parquet': 2643796},
 {'review-Kansas.parquet': 1927313},
 {'review-Kentucky.parquet': 1636239},
 {'review-Louisiana.parquet': 1485836},
 {'review-Maine.parquet': 1109348},
 {'review-Maryland.parquet': 2371154},
 {'review-Massachusetts.parquet': 2378534},
 {'review-Michigan.parquet': 2197036},
 {'review-Minnesota.parquet': 1778979},
 {'review-Mississippi.parquet': 1949236},
 {'review-Missouri.par

In [22]:
lista_ordenada = sorted(rows_to_analyze, key=lambda x: list(x.values())[0], reverse=True)

In [23]:
lista_ordenada

[{'review-Florida.parquet': 2729661},
 {'review-Connecticut.parquet': 2645648},
 {'review-Iowa.parquet': 2643796},
 {'review-New_York.parquet': 2638906},
 {'review-California.parquet': 2623809},
 {'review-Massachusetts.parquet': 2378534},
 {'review-Arkansas.parquet': 2374297},
 {'review-Maryland.parquet': 2371154},
 {'review-Pennsylvania.parquet': 2365164},
 {'review-Colorado.parquet': 2362828},
 {'review-Texas.parquet': 2234611},
 {'review-Oregon.parquet': 2222665},
 {'review-Indiana.parquet': 2219803},
 {'review-North_Carolina.parquet': 2204994},
 {'review-Michigan.parquet': 2197036},
 {'review-South_Carolina.parquet': 2075567},
 {'review-Idaho.parquet': 2057505},
 {'review-Illinois.parquet': 2052916},
 {'review-Mississippi.parquet': 1949236},
 {'review-Kansas.parquet': 1927313},
 {'review-New_Jersey.parquet': 1918931},
 {'review-Arizona.parquet': 1910655},
 {'review-Ohio.parquet': 1908238},
 {'review-Washington.parquet': 1904750},
 {'review-Georgia.parquet': 1894568},
 {'review-Alab